## Análisis vvvv
El objetivo ainvdsn

## Importe de librerías

In [1]:
import pandas as pd

## Análisis exploratorio de datos
Qué es lo que se pretende

Contexto
Definiciones de conceptos clave: 
Humedad, merma, etc... definiciones y cual es la unidad de medida

In [2]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas

In [3]:
file_path = 'data/Formato de Control de Calidad Café de Trillado.xlsx'
sheet_name_list = ['CONTROL CALIDAD CAFE TRILLADO J','Sheet2']

df_cc = pd.DataFrame()
# Cargar la hoja de Excel especificada, omitiendo las primeras 5 filas

columns = ['FECHA', 'LOTE', 'DENOMINACION_MARCA', 'CANTIDAD', 'HUMEDAD', 'HUMEDAD_C_NC', 'MALLAS', 'MALLAS_C_NC',
       'VERIFICACION_FISICA_CAFE_TOSTADO', 'NOTAS_CATACION', 'PUNTAJE',
       'PUNTAJE_C_NC', 'LIBERACION_LOTE', 'RESPONSABLE']

for sheet_name in sheet_name_list:
    df_temp = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=5)
    df_temp.columns = columns  # Asignar nombres de columnas
    df_cc = pd.concat([df_cc, df_temp], ignore_index=True)

df_cc.head(3)

,FECHA,LOTE,DENOMINACION_MARCA,CANTIDAD,HUMEDAD,HUMEDAD_C_NC,MALLAS,MALLAS_C_NC,VERIFICACION_FISICA_CAFE_TOSTADO,NOTAS_CATACION,PUNTAJE,PUNTAJE_C_NC,LIBERACION_LOTE,RESPONSABLE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,%,C/NC,#,C/NC,C/NC,NaN,N°,C/NC,SI/NO,NaN
2,19-07-22,01-190722,Madre Laura,765.0,10.9,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


In [4]:


#Eliminar filas con valores nulos en la columna 'DENOMINACION_MARCA'
df_cc = df_cc.dropna(subset=['DENOMINACION_MARCA'])
df_cc = df_cc.reset_index(drop=True)
df_cc

,FECHA,LOTE,DENOMINACION_MARCA,CANTIDAD,HUMEDAD,HUMEDAD_C_NC,MALLAS,MALLAS_C_NC,VERIFICACION_FISICA_CAFE_TOSTADO,NOTAS_CATACION,PUNTAJE,PUNTAJE_C_NC,LIBERACION_LOTE,RESPONSABLE
0,19-07-22,01-190722,Madre Laura,765.00,10.9,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,19-7-22,09-190722,Tabi Natural,204.00,10.2,C,14,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,19-07-22,10-190722,Don Mario,165.00,10.7,C,14,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,27-07-22,07-19-07-22,Don Felix,0.45,10.5,C,14,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,31-10-22,01-291022,Madre Laura,105.00,10.7,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2023-10-27 00:00:00,01-271023,Madre Laura,20.00,10.5,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,SI,AC
104,10-Ene-2024,01-100124,Madre Laura,20.00,10.4,c,14,c,c,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,SI,AC
105,02-Abril-2024,01-020424,Madre Laura,20.00,10.5,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,SI,AC
106,2024-06-20 00:00:00,01-200624,Madre Laura,14.00,10.8,C,14,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,SI,AC


In [5]:
df_cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   FECHA                             108 non-null    object 
 1   LOTE                              108 non-null    object 
 2   DENOMINACION_MARCA                108 non-null    object 
 3   CANTIDAD                          108 non-null    float64
 4   HUMEDAD                           108 non-null    object 
 5   HUMEDAD_C_NC                      108 non-null    object 
 6   MALLAS                            108 non-null    object 
 7   MALLAS_C_NC                       108 non-null    object 
 8   VERIFICACION_FISICA_CAFE_TOSTADO  108 non-null    object 
 9   NOTAS_CATACION                    107 non-null    object 
 10  PUNTAJE                           106 non-null    object 
 11  PUNTAJE_C_NC                      107 non-null    object 
 12  LIBERACI